## Package Imports

In [ ]:
import numpy as np
import pandas as pd

import os
import librosa

import scipy
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.svm import SVC

## Loading of Dataset & Metadata

In [ ]:
# Load data
audio_train_files = os.listdir('C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/dataset_train_unclean/')
audio_test_files = os.listdir('C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/dataset_test_svm/')

train = pd.read_csv('C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/metadata_train_unclean.csv')
submission = pd.read_csv('C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/metadata_submission.csv')

## Pre-processing

In [ ]:
SAMPLE_RATE = 44100

def clean_filename(fname, string):   
    file_name = fname.split('/')[1]
    if file_name[:2] == '__':        
        file_name = string + file_name
    return file_name

# Generate mfcc features with mean and standard deviation
def get_mfcc(name, path):
    data, _ = librosa.core.load(path + name, sr = SAMPLE_RATE)
    assert _ == SAMPLE_RATE
    try:
        ft1 = librosa.feature.mfcc(data, sr = SAMPLE_RATE, n_mfcc = 30)
        ft2 = librosa.feature.zero_crossing_rate(data)[0]
        ft3 = librosa.feature.spectral_rolloff(data)[0]
        ft4 = librosa.feature.spectral_centroid(data)[0]
        ft5 = librosa.feature.spectral_contrast(data)[0]
        ft6 = librosa.feature.spectral_bandwidth(data)[0]
        ft1_trunc = np.hstack((np.mean(ft1, axis = 1), np.std(ft1, axis = 1), skew(ft1, axis = 1), np.max(ft1, axis = 1), np.median(ft1, axis = 1), np.min(ft1, axis = 1)))
        ft2_trunc = np.hstack((np.mean(ft2), np.std(ft2), skew(ft2), np.max(ft2), np.median(ft2), np.min(ft2)))
        ft3_trunc = np.hstack((np.mean(ft3), np.std(ft3), skew(ft3), np.max(ft3), np.median(ft3), np.min(ft3)))
        ft4_trunc = np.hstack((np.mean(ft4), np.std(ft4), skew(ft4), np.max(ft4), np.median(ft4), np.min(ft4)))
        ft5_trunc = np.hstack((np.mean(ft5), np.std(ft5), skew(ft5), np.max(ft5), np.median(ft5), np.min(ft5)))
        ft6_trunc = np.hstack((np.mean(ft6), np.std(ft6), skew(ft6), np.max(ft6), np.median(ft6), np.max(ft6)))
        return pd.Series(np.hstack((ft1_trunc, ft2_trunc, ft3_trunc, ft4_trunc, ft5_trunc, ft6_trunc)))
    except:
        print('Bad file')
        return pd.Series([0] * 210)

In [ ]:
def convert_to_labels(preds, i2c, k = 3):
    ans = []
    ids = []
    for p in preds:
        idx = np.argsort(p)[::-1]
        ids.append([i for i in idx[:k]])
        ans.append(' '.join([i2c[i] for i in idx[:k]]))

    return ans, ids

## Feature Extraction

In [ ]:
train_data = pd.DataFrame()
train_data['fileName'] = train['fileName']
test_data = pd.DataFrame()
test_data['fileName'] = audio_test_files

train_data = train_data['fileName'].progress_apply(get_mfcc, path = 'C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/dataset_train_unclean/')
print('Finished loading MFCC for training dataset')
test_data = test_data['fileName'].progress_apply(get_mfcc, path = 'C:/Users/Jerome/Desktop/Jerome/Programmierung Projekte/Thesis-Projekt/ACS-master-final/backend/data/dataset_test_svm/')
print('Finished loading MFCC for testing dataset')

train_data['fileName'] = train['fileName']
test_data['fileName'] = audio_test_files

train_data['className'] = train['className']
test_data['className'] = np.zeros((len(audio_test_files)))

In [ ]:
X = train_data.drop(['className', 'fileName'], axis = 1)
feature_names = list(X.columns)
X = X.values
labels = np.sort(np.unique(train_data.className.values))
num_class = len(labels)
c2i = {}
i2c = {}
for i, c in enumerate(labels):
    c2i[c] = i
    i2c[i] = c
y = np.array([c2i[x] for x in train_data.className.values])

In [ ]:
X_test = test_data.drop(['className', 'fileName'], axis = 1)
X_test = X_test.values

In [ ]:
# Apply scaling for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Apply PCA for dimension reduction
pca = PCA(n_components = 65).fit(X_scaled)
X_pca = pca.transform(X_scaled)
X_test_pca = pca.transform(X_test_scaled)

## SVM Training

In [ ]:
# Fit an SVM model
X_train, X_val, y_train, y_val = train_test_split(X_pca, y, test_size = 0.2, random_state = 42, shuffle = True)

clf = SVC(kernel = 'rbf', probability = True)

clf.fit(X_train, y_train)
print('Primary Training Accuracy: ', accuracy_score(clf.predict(X_train), y_train) * 100, '%')
print('Primary Testing Accuracy: ', accuracy_score(clf.predict(X_val), y_val) * 100, '%')

In [ ]:
C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1, 10]
param_grid = {'C': C_grid, 'gamma' : gamma_grid}

grid = GridSearchCV(SVC(kernel = 'rbf'), param_grid, cv = 3, scoring = "accuracy")
grid.fit(X_train, y_train)

In [ ]:
# Optimal model
clf = SVC(kernel = 'rbf', C = 4, gamma = 0.01, probability = True)

history = clf.fit(X_train, y_train)

print('Actual Training Accuracy: ', accuracy_score(clf.predict(X_train), y_train) * 100, "%")
print('Actual Testing Accuracy: ', accuracy_score(clf.predict(X_val), y_val) * 100, "%")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

X, y = make_classification(random_state = 65)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 65)
clf = SVC(random_state = 65)
clf.fit(X_test, y_test)
SVC(random_state = 65)
plot_confusion_matrix(clf, X_train, y_train)  
plt.title("SVM Confusion Matrix")
plt.show()